In [4]:
import csv
import numpy as np
from numba import vectorize, cuda, int32


In [5]:
@vectorize( 'int64(int64, int64)', target='cuda')
def Add(a, b):
  return a + b

### Experiment 1

In [13]:
data = {}
features = np.full((610, 9724), -1)
with open('ratings.csv') as csv_file:
    data = csv.DictReader(csv_file)
    print(type(data))
    mov_index = {}
    index = 0
    for i in data:
        if i['movieId'] not in mov_index.keys():
            mov_index[i['movieId']] = index
            index += 1
        if float(i['rating']) >= 3:
            features[int(i['userId'])-1][mov_index[i['movieId']]] = 1
print(index)
print(features[0][mov_index['1']])

        

<class 'csv.DictReader'>
9724
1


### Experiment 2

In [8]:
train_features = features[:549]
test_features = features[549:]

The network should be able to store all 549 users in the trainning set so the netork which will have 9724 nodes should be able to store all of them

### Experiment 3

In [14]:
#compute weights in symmetric matrix
#entries along the diagonal are the bias
def train_hopfield(features):
    index = 0;
    weights = np.zeros((9724, 9724), dtype= np.dtype(np.int64))
    
    for i in features:
        temp_w = np.outer(i, i)
      
             
        for j in range(9724):
            temp_w[j][j] += i[j]
        weights = np.add(weights, temp_w)
    return weights
    

In [10]:
def remember(feature, weights):
    stable = False
    epochs = 0
    
    while not stable:
        num_stable = 0
        stable = True
        x = feature
        #x = np.random.shuffle(feature)
        for i in range(len(feature)):
            last = feature[i]
            update_val = 0 #weights[i][i] #add bias to update
            for j in range(len(feature)):
                if j != i:
                    update_val += weights[i][j] * feature[j]
            if update_val >= 0:  
                feature[i] = 1
            else:
                feature[i] = -1
            if feature[i] !=  last:
                stable = False
            else: 
                num_stable += 1
        print("epochs = ", epochs)
        print("stable = ", num_stable)
        epochs+=1
    return feature           

In [14]:
@cuda.jit
def update(result, weights, features):
 
    index = cuda.grid(1)
    temp = 0
    while index < len(features):
        temp += features[index] * weights[result[1]][index]
      
        cuda.atomic.add(result, 0, temp)
        index = index + cuda.gridsize(1) 

        
        

In [116]:
#testing updating all features at once using one block of threads to update each 
@cuda.jit
def remember_device1(features, weights, stable):
    y = cuda.blockIdx.x
    x = cuda.threadIdx.x
    last = features[y] 
    update = cuda.shared.array((2), np.int32) #one used as stable the other is the update value
   
    if x == 0:
       update[0] = 0
    cuda.syncthreads()
    temp = 0
    while x < len(features):
        temp += features[x] * weights[y][x]
    
        cuda.atomic.add(update, 0, temp)
        x = x + cuda.blockDim.x
    cuda.syncthreads()
    if x == 0 :
        if update[0] >= 0:
            features[y] = 1
        else:
            features[y] = -1
        if last ==  features[y]:
            cuda.atomic.add(stable, 0, -1)
    


In [120]:
def exp_remember(feature, weights):
    stable = np.array([1])
    while stable > 0:
        stable[0]= len(feature)
        grid_dim = len(feature)
        remember_device1[9724, 512](feature, weights, stable)
    return feature

In [21]:
#parralel remember
def remember_par(feature, weights):
    stable = False
    epochs = 0
    recommended = np.full(9724, -1)   
    while not stable:
        num_stable = 0
        stable = True
        x = feature
        
        #x = np.random.shuffle(feature)
        for i in range(len(feature)):
            last = feature[i]
            update_val = 0 #weights[i][i] #add bias to update
#             for j in range(len(feature)):
#                 if j != i:
#                     update += weights[i][j] * feature[j]
            d_weights = cuda.to_device(weights) 
            print(i)
            result = np.array([update_val, i], dtype= np.dtype(np.int32))
           
            num_threads = 512
            num_blocks = 18
            update[num_blocks, num_threads](result, d_weights, feature )
            update_val = result[0]
            if update_val >= 0:  
                feature[i] = 1
            else:
                feature[i] = -1
            if feature[i] !=  last:
                stable = False
            else: 
                num_stable += 1
        print("epochs = ", epochs)
        print("stable = ", num_stable)
        epochs+=1
        
      
    return feature           

In [15]:
 weights = train_hopfield(train_features)
 

In [37]:
 recommended = remember(train_features[0], weights)

epochs =  0
stable =  9497
epochs =  1
stable =  9724


In [38]:
print(train_features[0])

[-1 -1 -1 ... -1 -1 -1]


In [31]:
if False not in np.equal(recommended, train_features[0]):
    print('sucess')
else:
    print('failure')

sucess


In [32]:

correct = 0
for x in train_features:
    recommended = remember(x, weights)
    if False not in np.equal(recommended, x):
        correct += 1
accuracy =  correct/len(train_features)
print(accuracy)

epochs =  0
stable =  9497
epochs =  1
stable =  9724
epochs =  0
stable =  9698
epochs =  1
stable =  9724
epochs =  0
stable =  9705
epochs =  1
stable =  9724
epochs =  0
stable =  9556
epochs =  1
stable =  9724
epochs =  0
stable =  9685
epochs =  1
stable =  9724
epochs =  0
stable =  9431
epochs =  1
stable =  9724
epochs =  0
stable =  9619
epochs =  1
stable =  9724
epochs =  0
stable =  9682
epochs =  1
stable =  9724
epochs =  0
stable =  9689
epochs =  1
stable =  9724
epochs =  0
stable =  9613
epochs =  1
stable =  9724
epochs =  0
stable =  9666
epochs =  1
stable =  9724
epochs =  0
stable =  9691
epochs =  1
stable =  9724
epochs =  0
stable =  9695
epochs =  1
stable =  9724
epochs =  0
stable =  9683
epochs =  1
stable =  9724
epochs =  0
stable =  9622
epochs =  1
stable =  9724
epochs =  0
stable =  9631
epochs =  1
stable =  9724
epochs =  0
stable =  9620
epochs =  1
stable =  9724
epochs =  0
stable =  9257
epochs =  1
stable =  9724
epochs =  0
stable =  9366
e

KeyboardInterrupt: ignored

In [36]:
my_movies = np.full(9724, -1)
my_movies[mov_index['1']] = 1 #Toy Story
my_movies[mov_index['2']] = 1 #Jumanji
my_movies[mov_index['95']] = 1 #Broken Arrow
my_movies[mov_index['46972']] = 1 #Night at the Musuem
my_movies[mov_index['179401']] = 1 #Jumanji: Welcome to the Jungle
print(my_movies[0])
my_recommendations = remember(my_movies, weights)
print(my_movies[0])
recommended_to_me = np.equal(my_movies, my_recommendations)
index = 0
pr = 0
while pr < 5 and index <  9724:
    if  my_recommendations[index] != my_movies[index]:
        key_list = list(mov_index.keys())
        val_list = list(mov_index.values())
        print('recommendation 1 =', key_list[vallist.index(index)])
        pr +=1
    index += 1

1
epochs =  0
stable =  9718
epochs =  1
stable =  9724
-1


In [35]:
print(my_movies[0])

-1
